### predictions

### import modules and configure notebook

In [1]:
import pandas as pd
import numpy as np
import swifter
import seaborn as sns
import matplotlib.pyplot
import pickle

from functions.model_data import bestHyperparamaters 
from functions.postproccessing import process_results

pd.set_option('max.rows', None)
pd.set_option('max.columns', None)
%matplotlib inline

### Load variables stored by data_preproccessing notebook

In [2]:
%store -r train_data_formodel
%store -r test_data
%store -r my_data
%store -r uniques
%store -r best_feats
%store -r X_test_labeled_df



In [3]:
X_test_labeled_df.head()

Analysis  Zr90  Ba137  Sr88  Ge72  Cr52     S33      U238         Al27  \
0  06_DH1_1  1.43   5.95  2.00  1.37  5.18  806.55  0.250083   635.240000   
1  07_DH1_2  1.47   7.51  1.84  1.93  5.43  807.55  0.250083   639.090000   
2  08_DH1_3  1.16   4.40  1.93  2.14  2.61  837.73  0.250083   670.480000   
3  09_DH2_1  1.45  12.48  4.29  1.59  2.87  750.62  0.080000  1515.220000   
4  10_DH2_2  1.99  14.40  4.44  0.81  5.92  796.21  0.090000   710.920579   

     B11   Mg24  Nd146  Sc45         K39  Pr141    Li7  inlierLabel  
0  93.21  27.83   0.34  1.08  443.100000   0.07   2.79            1  
1  95.19  27.99   0.24  1.09  442.100000   0.06   2.37           -1  
2  93.63  26.43   0.08  1.17  444.280000   0.03   2.46           -1  
3  63.80  31.98   0.25  1.19  369.135675   0.06  18.47            1  
4  60.96  44.95   0.51  1.22  369.135675   0.12  19.98            1

In [4]:
train_data_formodel.head()

Analysis  Geology  Province Region Site SubSite Formation Band   Nodule  \
0  10_FH1_1_1  Bedrock  Northern    NaN   FH     FH1   Burnham  FH1  FH1_1_1   
1  11_FH1_1_1  Bedrock  Northern    NaN   FH     FH1   Burnham  FH1  FH1_1_1   
2  12_FH1_1_1  Bedrock  Northern    NaN   FH     FH1   Burnham  FH1  FH1_1_1   
3  13_FH1_1_2  Bedrock  Northern    NaN   FH     FH1   Burnham  FH1  FH1_1_2   
4  14_FH1_1_2  Bedrock  Northern    NaN   FH     FH1   Burnham  FH1  FH1_1_2   

     Li7   Be9    B11        Mg24     Al27       Si28          P31      S33  \
0  15.63  0.12  48.36  154.630000   943.71  464944.18    50.280000   538.57   
1  11.50  0.09  44.77   43.735431  1077.11  465010.94    70.910000   438.20   
2  20.05  0.06  44.88   42.700000   620.21  465295.41   104.470000   372.66   
3  11.16  0.73  47.06  162.420000  1143.19  402596.61  3735.154885  1075.89   
4  17.71  0.32  48.26   33.520000   547.22  465027.11    44.440000   464.78   

      K39     Ca42  Sc45   Ti47   V51    Cr52  Mn55    Fe56  Co59  Ni60  Cu63  \
0  455.94   712.39  0.42  15.58  0.27    3.30  0.69    8.46  0.05  0.80  1.62   
1  387.82   515.24  0.44  18.47  0.29    3.45  1.01   11.59  0.11  0.36  0.53   
2  363.71   957.89  0.76  19.89  0.55    3.25  1.21   87.99  0.21  1.68  1.53   
3  547.55  2174.30  0.43  42.30  0.67  152.42  4.84  145.34  0.30  2.45  5.02   
4  278.25  1551.63  0.71  11.18  0.27    2.56  1.73   25.38  0.05  0.80  0.55   

    Zn68  Ga69  Ge72  As75  Rb85   Sr88   Y89  Zr90  Nb93  Mo95  Cd111  In115  \
0  10.82  0.25  1.22  0.16  0.43  12.94  0.88  1.51  0.09  0.05   0.02   0.00   
1   8.93  0.34  0.85  0.10  0.45  13.22  0.95  1.74  0.07  0.01   0.02   0.00   
2  11.98  0.25  1.71  0.13  0.43   8.52  0.87  0.93  0.10  0.02   0.02   0.00   
3  17.15  0.35  2.13  0.84  0.76  13.16  0.97  2.00  0.10  0.29   0.18   0.01   
4   9.80  0.41  1.41  0.12  0.28   9.90  0.90  0.90  0.08  0.04   0.10   0.00   

   Sn118  Cs133  Ba137  La139  Ce140  Pr141  Nd146  Sm147  Eu153  Gd157  \
0   0.05   0.01   6.54   0.84   0.95   0.23   0.87   0.16   0.04   0.16   
1   0.04   0.02   8.04   0.92   1.01   0.23   0.98   0.18   0.04   0.18   
2   0.05   0.01   3.13   0.90   1.08   0.26   0.84   0.15   0.04   0.19   
3   0.78   0.04   8.74   0.93   0.95   0.21   0.75   0.13   0.04   0.25   
4   0.09   0.01   2.74   0.97   1.09   0.27   1.00   0.17   0.04   0.19   

   Tb159  Dy163  Ho165  Er166  Tm169  Yb172  Lu175  Hf178  Ta181  Pb208  \
0   0.02   0.11   0.03   0.06   0.01   0.02   0.00   0.04   0.01   0.24   
1   0.02   0.13   0.03   0.06   0.01   0.04   0.01   0.05   0.00   0.07   
2   0.02   0.14   0.02   0.07   0.01   0.06   0.00   0.02   0.01   0.46   
3   0.02   0.09   0.03   0.05   0.00   0.03   0.00   0.08   0.00   0.64   
4   0.02   0.15   0.03   0.05   0.01   0.05   0.01   0.02   0.01   0.59   

   Th232  U238  class  
0   0.07  0.05      0  
1   0.08  0.04      0  
2   0.05  0.05      0  
3   0.05  0.03      0  
4   0.06  0.09      0

### configurations

* scale -> True|False if set to True then features scaled to all have mean value 0 and standard deviation 1

In [5]:
save_predictions = True

In [6]:
from config_and_dependencies.config import *

### counts of instances in all classes before oversampling

In [7]:
train_data_formodel['class'].value_counts()

4     105
15    100
16     61
0      53
11     45
13     36
14     36
2      36
10     30
7      30
6      30
5      27
8      27
1      24
12     21
3      18
9      17
Name: class, dtype: int64

### The class column is stored as the variable y and the variables identified as best by the 2 feature_selection notebook are used as features

In [8]:
y = np.array(train_data_formodel[target])
X = np.array(np.array(train_data_formodel[best_feats]))

### the dimensions of the class and features are checked

In [9]:
print(X.shape)
print(y.shape)

(696, 15)
(696,)


### grid search is utilised to identify optimum hyperparamaters

In [10]:
my_optimiser = bestHyperparamaters(X, y)
my_optimiser.get_best_model()

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   22.0s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  5.3min
C:\Users\tom\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed: 10.7min finished


In [11]:
best_model = my_optimiser.best_model

### Model is built for predicting source of artefacts 

In [12]:
best_model.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=16, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=2400,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

# sanity check to see model correctly predicts geological samples

In [13]:
train_check = best_model.predict(X)
sum(train_check == y)/len(y)

1.0

In [14]:
train_check == y

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [15]:
train_check

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4

In [16]:
X.shape

(696, 15)

# When I check the encodings below with that in notebook 1, it is to be correct

In [17]:
train_check

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4

In [18]:
print(best_model)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=16, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=2400,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)


In [19]:
import pickle


In [20]:
filename = 'output_datasets/models/rfc_model.sav'
pickle.dump(best_model, open(filename, 'wb'))


FileNotFoundError: [Errno 2] No such file or directory: 'output_datasets/models/rfc_model.sav'

In [ ]:
my_dict = {}
numbers = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16]
sites = ['FH', 'ER', 'WW', 'TC', 'CS', 'KQ', 'AR', 'SL', 'FG', 'WB', 'PF', 'WH',
       'SQ', 'WN', 'BH', 'PH', 'LB']

In [ ]:
for i in numbers:
    my_dict[i] = sites[i]

In [ ]:
my_dict

In [ ]:
print(train_data_formodel[target].unique())
print(uniques)

In [ ]:
identifiers =  X_test_labeled_df['Analysis']

In [ ]:
X_test_labeled_df.head()

In [ ]:
results = process_results(best_model, X_test_labeled_df, best_feats, uniques, identifiers)

# test

In [ ]:
y_pred = best_model.predict(np.array(X_test_labeled_df[best_feats]))

In [ ]:
X_test_labeled_df

In [ ]:
y_pred

In [ ]:


y_pred = best_model.predict(np.array(data[best_feats]))
y_pred_proba = best_model.predict_proba(np.array(data[best_feats]))

probabilities_df = pd.DataFrame(data = y_pred_proba, columns = uniques)
probabilities_df_final = pd.concat([probabilities_df, identifiers.reset_index(drop = True)], axis = 1)

final_pred_df = pd.concat([pd.Series(y_pred), probabilities_df_final], axis = 1).rename(columns={0:'class_number'})

final_predictions_df = pd.concat([final_pred_df, data['inlierLabel']], axis = 1)

uniques_list = list(uniques)
def get_pred_names(row):
    return(uniques_list[row['class_number']])

final_predictions_df['class_predictions'] = final_predictions_df.apply(get_pred_names, axis = 1)

def outlierAssigner(row):
    if row['inlierLabel'] == -1:
        return('other')
    else:
        return(row['class_predictions'])

final_predictions_df['class_predictions'] = final_predictions_df.swifter.apply(outlierAssigner, axis = 1)

# test

In [ ]:
results

### I think if there is a problem here it's most likely to be in the encodings of the class, perhaps they are not correct?

In [ ]:
results['class_number'].value_counts()

In [ ]:

#filter for each class prediction, for each class prediction average the probabilities then plot as bar plot, check dists, then filter for outlier or not outlier and bar chart the average probabilities again

### predictions are stored as a variable into memory

In [ ]:
%store results

### predictions are outputted as csv file

In [ ]:
if save_predictions:
    results.to_csv('output_datasets/predictions.csv')


In [ ]:
results.class_predictions.value_counts()